# Classification

In [1]:
import numpy as np

## 单量指标
| 真实结果 | 预测结果 | 预测结果 |
| ----- | --------- | ----------- |
|    | 真 | 假 |
| 真 | TP | FP |
| 假 | FN | TN |

$$
\begin{align}
Accuracy &= \frac {TP+TN}{TP+FP+FN+TN}\\ \\
Precision &= \frac {TP}{TP+FP}\\ \\
Recall &= \frac {TP}{TP+FN}\\ \\
F_1 &= \frac {2\times Precision \times Recall}{Precision + Recall}
\end{align}
$$

In [4]:
def singleMetics(P, R):
    TP = ((P==1) & (R==1)).sum()
    FP = ((P==0) & (R==1)).sum()
    FN = ((P==1) & (R==0)).sum()
    TN = ((P==0) & (R==0)).sum()
    Accuracy = (TP+TN)/len(P)
    Precision = TP/(TP+FP)
    Recall = TP/(TP+FN)
    F1 = (2*Precision*Recall)/(Precision+Recall)
    return { 'TP':TP, 'FP':FP, 'FN':FN, 'TN':TN, 'Accuracy':Accuracy,
             'Precision':Precision, 'Recall':Recall, 'F1':F1}

## P-R曲线
以 Precision 为 Y 轴，Recall 为 X 轴的曲线